## What are decorators? How to use them

Decorators provide a simple syntax for calling high order functions. They extend the behaviour of a function without explicitly modifying it. 

## Brief functions notes

A function may have a side effect when aside from turning an input into an output, they perform another action that has some effect on the system. For example print() not only returns None but also prints out to stdout. (notice how the output is not necesarely the desired action).

In **functional programming**, you work almost only wuth *pure* functions, meaning they have no side effects.

### Functions can be passed around and used as arguments, because they are first-class objects. 

In [2]:
def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

print(greet_bob(say_hello))
print(greet_bob(be_awesome))

Hello Bob
Yo Bob, together we are the awesomest!


### Inner functions
- Functions can have other functions declared inside their body. These inner or child functions will not be declared until the outer or parent function is executed.
- You can return inner function definitions from the parent. 

## Decorator example

In [3]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper


def say_whee():
    print("Whee!")


say_whee = my_decorator(say_whee) # decoration

The core function is passed as a parameter to the decorator, that then wraps the function call in other statemets that belong to the function wrapper, then this wrapper is returned as a function. What you get is the package function+decorator in a single function. 

In [4]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


We can then say that Deocorators *wrap* a function, without modifying its behaviour.

In [5]:
from datetime import datetime


def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass  # Hush, the neighbors are asleep
    return wrapper


def say_whee():
    print("Whee!")


say_whee = not_during_the_night(say_whee)

In [6]:
say_whee()

Whee!


## Syntactic Sugar

In [7]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper


@my_decorator
def say_whee():
    print("Whee!")


In [8]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


### Decorating functions with arguments

If we want to pass an argument to the decorated function, the example used so far will raise an error, becaouse the decorator definition does not pass any arguments to the function. 

We fix this by allowing any number of arguments be passed to a decorated function inside the decorator definition. 


In [9]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice


@do_twice
def greet(name):
    print(f"Hello {name}")

greet("john")

Hello john
Hello john


### Return values

How do we return a value from a decorated function? It's up to the decorator to decide what will be returned. It can modify the return value of the wrapper thus modifying the original expected value. In the example above, the decorator wrapper does not return any value from the function called, so if we would choose to decorate a function with a return value, the wrapper would *eat* the returned value.

In [11]:
@do_twice
def greet(name):
    return f"Hello {name}"

print(greet("john"))


None


How do we fix this?

In [12]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice


@do_twice
def greet(name):
    return f"Hello {name}"

print(greet("john"))

Hello john


In [16]:
def print_twice(func):
    def wrapper_print_twice(*args, **kwargs):
        return f'{func(*args, **kwargs)}\n{func(*args, **kwargs)}'
    return wrapper_print_twice

@print_twice
def greet(name):
    return f"Hello {name}"

print(greet("john"))


Hello john
Hello john


All functions have an instrospection ability that allows them to report information about them.

In [17]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [18]:
help(greet)

Help on function wrapper_print_twice in module __main__:

wrapper_print_twice(*args, **kwargs)



But, the decorated function has some kind of an identity crisis, and doesn't really know who it is. To fix this, we use functools.wrap decorator, which preserves information about the original function.

In [22]:
import functools


def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def greet(name):
    return f"Hello {name}"

help(greet)


Help on function greet in module __main__:

greet(name)



### Template for decorators

In [ ]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator


### Debugger decorator

In [24]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]                      # 1
        kwargs_repr = [f"{k}={v!r}" for k, v in kwargs.items()]  # 2
        signature = ", ".join(args_repr + kwargs_repr)           # 3
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__!r} returned {value!r}")           # 4
        return value
    return wrapper_debug


One can apply a decorator to an already defined function using the original decorating assign:

In [26]:
import math

# Apply a decorator to a standard library function
math.factorial = debug(math.factorial)


def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

approximate_e(5)

Calling factorial(0)
Calling factorial(0)
'factorial' returned 1
'factorial' returned 1
Calling factorial(1)
Calling factorial(1)
'factorial' returned 1
'factorial' returned 1
Calling factorial(2)
Calling factorial(2)
'factorial' returned 2
'factorial' returned 2
Calling factorial(3)
Calling factorial(3)
'factorial' returned 6
'factorial' returned 6
Calling factorial(4)
Calling factorial(4)
'factorial' returned 24
'factorial' returned 24


2.708333333333333

### Registering functions as plugins

This is an example of a decorator that does not wrap a function, rather it does something else (in this case, register the function in a plugins list) and then return the function unmodified. 

In [28]:
import random
PLUGINS = dict()


def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func


@register
def say_hello(name):
    return f"Hello {name}"


@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"


def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

randomly_greet('Corina')


Using 'be_awesome'


'Yo Corina, together we are the awesomest!'

This removes the necessity to create a maintain a plugins list of which plugins exist. This is similar to globals() in python, like a namespace holder for functions. 

# More on decorators

### Decorators on classes

- You can apply decorators on class methods
- There are common decorators for class methods like @property, @classmethod, @staticmethod
- Appling common decorators to class definitions may not have the same behaviour as with functions or methods. A class decorator does not apply the decorator to all it's methods. 
- A @singleton decorator is a class decorator

In [ ]:
class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535


### Nested decorators

- one can call one decorator before the other and they will be applied in the order that they were called.

### Decorators with arguments

In [33]:
def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for i in range (0,num_times): # num_times must be supplied from outside, so a closure is created
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat

@repeat(num_times=4)
def greet(name):
    print(f"Hello {name}")

greet('john')


Hello john
Hello john
Hello john
Hello john


### Decorators that can be used both with and without arguments

Using the special character *, we can specify that all following parameters of a function are keyword only.

In [ ]:
def repeat(_func=None, *, num_times=4):  # 1
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat                      # 2
    else:
        return decorator_repeat(_func)               # 3


### Stateful Decorators

Decorators that can keep track of a state. For example for counting the times that the function was called

In [ ]:
import functools


def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__!r}")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls


@count_calls
def say_whee():
    print("Whee!")


### Classes as Decorators

It's the typical way to maintain state. In order to make a class as a decorator, one must take a function as a parameter in the __init__() method, so as to return a function when the class is instanciated. In typical class implementation, the __call__() method is executed each time an instance is called. A typical class decorator implements these two methods.

Class stateful decorator

In [ ]:
import functools


class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func) # we use this rather than ordinary 'wraps' to allow stateful behaviour
        self.func = func    # Store a reference to the function
        self.num_calls = 0  # Other necessary initializations

    def __call__(self, *args, **kwargs):  # This is what is called instead of the decorated function.
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)


@CountCalls
def say_whee():
    print("Whee!")
